In [78]:
import os
import pandas as pd
import fnmatch
import re

In [79]:
# BIGGO TO DO:
# trash bin for each filter

In [80]:
# NOTE: Possible names for each sequence

# "3D_T1": ['*SAG*T1*NII', '*SAG*FSPGR*3D*NII']
# "FLAIR": <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
# "BOLD": <ID>_BOLD_<DOE>.nii
# "Diffusion kurtosis": <ID>_{AX}_DkI_<small#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bvec
# .                     <ID>_{AX}_DkI_<big#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bvec
# "Diffusion tensor": <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.nii
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bval
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bvec
# "Diffusion weithed": <ID>_{AX}_DWI_<DOE>.nii
#                      <ID>_{AX}_DWI_<DOE>.bval
#                      <ID>_{AX}_DWI_<DOE>.bvec
# "ASL": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii   
# "SWAN": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii
# "Carotid"
# "Aorta": <ID>_[CINE]_AORTA_VENC_<number>_<DOE>.nii

In [81]:
def get_sequence_info(sequence_name):
    """
    Convert the input from string to list 
    --------------------------------------------------------
    input:
        sequence_name: string
        
    return:
        array-like: splited the string into items for parsing later
    """
    str_list = sequence_name.replace('-','_').replace('.','_').split('_')[:-1]
    str_list = list(filter(None, str_list)) 
    return str_list

#selected everything expect the '.nii' part of the string

In [82]:
def ignore_notes(sequence_info):
    """
    ignore notes inside the parentheses of sequence_info
    --------------------------------------------------------
    input:
        sequence_info: array-like
        
    return:
        array-like: sequence info without the parentheses and notes inside of them
    """
    
    other = []
    
    for part in sequence_info:
        if '(' not in part and ')' not in part:
            other.append(part)

    return other

In [83]:
def get_DOE(info):
    """
    get the date of exam 
    --------------------------------------------------------
    input:
        info: array-like
        
    return:
        string: date of exame in YYYY/MM/DD
    """
    
    if 'I' in info[-1]:
        DOE=info[-2]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])
    else:
        DOE=info[-1]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])

In [84]:
def base_standard(o_name):
    """
    clean up the user input sequence name into a standardized naming convetion for sorting later
        SAGITTLE --> SAG
           AXIAL --> AX
         CORONAL --> COR
            CUBE --> 3D
        WHITTIER --> WH
              RT --> RIGHT
              LT --> LEFT
    --------------------------------------------------------
    input:
        o_name: string
    
    return:
        string: standardized sequnece file name
    """
    
    n_name = o_name.replace('SAGITTAL', 'SAG').replace('CUBE', '3D').replace('AXIAL', 'AX').replace('CORONAL', 'COR')
    n_name = n_name.replace('RT', 'RIGHT').replace('LT', 'LEFT')
    info=get_sequence_info(n_name)
    
    ID = info[0]
    # ID does not follow designation [2-3 letter cohort designation][3-4 number designation]
    match = re.match(r"([A-Z]+)([0-9]+)", ID, re.I)
    if match:
        parts = match.groups()
        if parts[0] == 'WHITTIER':
            n_name = n_name.replace('WHITTIER', 'WH')
    return n_name

In [85]:
def filt_T1(file_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    # TO-DO: how to handle 
    # 1) NEUROMELANIN scans
    # 2) SE_MT
    # 3) BRAVO
    # 4) 2DTOF
    # 5) repeat?
    
    sequence_info=get_sequence_info(file_name)
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None

    ignore=['AVG', 'MIN', 'POST_PROCESS', 'MT', '2DTOF', 'OBLIQUE', 'THIN']
    if any(word in file_name for word in ignore):
        return None
    
    accept = ['SAG', 'AX', 'COR']
    if not any(word in file_name for word in accept):
        return None
    
#     print("file name:", file_name)
    return file_name[:-4]+'.nii'

    
#TO-DO: repeated exam of the same date or at the different date
    

In [86]:
# Naming to be estabilshed
# <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_[MIP_ | MIN_IP_]<DOE>[_<tag>].nii
def filt_FLAIR(sequence_name, flair_3d, flair_2d):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
            
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None
    
    if "SAG" in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'AX' in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'COR' in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    else: 
#         print("\t ", sequence_name)
        return None
            
    sequence_name = '_'.join(sequence_info)+'.nii'
    return sequence_name

#TO-DO: T1 has 3D, 2D

In [87]:
# Naming to be established:
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
def filt_T2(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
    
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None
#     print(sequence_info)
        
    return '_'.join(sequence_info)+'.nii'

#TO-DO: T2 has difference pulse types :0 and some other things

In [88]:
def filt_diffusion(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)

#TO-DO: this one will be A LOT OF FUN!

In [89]:
# BOLD naming convention: <ID>_BOLD_<DOE>.nii
def filt_bold(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
    
    problem_bold = None
    
    sequence_info = ignore_notes(sequence_info)
    sequence_name = '_'.join(sequence_info)+'.nii'    
    
    # The following ID's are the problem ones (from previous runs). Checking if these ID's are unique
#     probs = ['SH005', 'WH1296', 'MEN009', 'WH1612', 'WH1268', 'MEN040', 'SH019', 
#              'MEN023', 'MEN007', 'WHITTIER5002']
    
    # If it contains EQ_1 or a letter attached to the end of the DOE,
    # then there is a previous one that already exists
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        return None
        
#     elif ID in probs:
#         problem_bold = sequence_name
    
    elif len(sequence_info) != 3:
        problem_bold = sequence_name
    
#     return problem_bold
    return sequence_name

In [90]:
def filt_asl(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)            
    sequence_info=ignore_notes(sequence_info)
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None

    ignore=['DELAY', 'FL']
    if any(word in sequence_name for word in ignore):
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [91]:
def filt_swan(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)            
    sequence_info=ignore_notes(sequence_info)
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        return None

    ignore=['MIDBRAIN']
    if any(word in sequence_name for word in ignore):
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [92]:
def filt_csf(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    if '1.5' in sequence_name:
        return None
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)
    if not sequence_info[-1].isdigit():
        return None
    
    # NOTE: V vs VC vs VENC
    ignore=['CF', 'FM']
    if any(word in sequence_name for word in ignore):
        return None
    
    return '_'.join(sequence_info)+'.nii'

#TO-DO: CSF of different location and CINE ON/OFF

In [93]:
def filt_carotid(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)

    if not sequence_info[-1][-1].isdigit():
        return None 
    
    print(sequence_info)

In [94]:
def filt_aorta(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)

    ignore=['ASCENDING']
    if any(word in sequence_name for word in ignore):
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [95]:
def filt_candy_cane(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_info)
    
    if not sequence_info[-1].isdigit():
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [96]:
def filt_arch(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(info)
    if not info[-1].isdigit():
        return None
    
    if 'VENC' not in info:
        info.insert(3, 'VENC_???')
    
    return '_'.join(info)[:-4]+'.nii'

In [97]:
def filt_mpr(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [98]:
def filt_cbf(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        return None
    return '_'.join(sequence_info)+'.nii'

In [99]:
def filt_probe(sequence_name):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)
    return '_'.join(sequence_info)+'.nii'

In [100]:
def get_path(root):
    """
    Fliter through all the files in the given path
    --------------------------------------------------------
    Parameters:
        root : string
            Path the directory that contains all the files that is to be filtered through
    """
    
    t1=[]
    flair = []
    flair_3d = []
    flair_2d = []
    t2 = []
    bold = []
    swan = []
    asl=[]
    candy_cane=[]
    arch=[]
    aorta=[]
    csf = []
    cbf=[]
    probe = []
    
    change = 0
    count = 0
    
    for path, subdirs, files in os.walk(root):
        for file in files:
            file_upper=base_standard(file.upper())
            if fnmatch.fnmatch(file_upper, '*T1*NII') or fnmatch.fnmatch(file_upper, '*FSPGR*NII'):                 
                n_file=filt_T1(file_upper)
#                 print("T1:", n_file)
                if n_file is not None:
                    t1.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*FLAIR*NII'): 
#                 print("FLAIR: \t", file_upper)
                n_file = filt_FLAIR(file_upper, flair_3d, flair_2d)
#                 print("FLAIR:", n_file)
                if n_file is not None:
                    flair.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*T2*NII'):
#                 print('\t', file_upper)
                n_file = filt_T2(file_upper)
                if n_file is not None:
                    t2.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*BOLD*NII'):
                n_file = filt_bold(file_upper)
                if n_file is not None:
#                     print(file, '-->', n_file)
                    bold.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*DTI*'):
#                 print('\t', file_upper)
#                 sort_diffusion(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*DKI*') or fnmatch.fnmatch(file_upper, '*DTI*'):
                pass
            elif fnmatch.fnmatch(file_upper, '*ASL*NII'):
                n_file=filt_asl(file_upper)
                if n_file is not None:
                    asl.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*SWAN*NII'):
                # Swan files has very consistant sequence naming
                # beaware of ID naming though
                n_file = filt_swan(file_upper)
                if n_file is not None:
                    swan.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*CSF*NII'):
#                 print('CSF:', file_upper)
                n_file = filt_csf(file_upper)
                if n_file is not None:
                    csf.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*CAROTID*NII'):
                #Need to ask what the carotid is doing: 
                #1. need to know the naming convention for difference carotids
#                 print('Carotid:', file_upper)
                filt_carotid(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*AORTA*NII'):
                n_file=filt_aorta(file_upper)
                if n_file is not None:
                    aorta.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*CEREBR*BLOOD*FLOW*NII'):
#                 print("cbf:", file_upper)
                n_file=filt_cbf(file_upper)
                if n_file is not None:
                    cbf.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                     print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*MPR*NII'):
#                 print(file_upper)
#                 filt_mpr(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*CANDY_CANE*NII'):
#                 print('Candy cane:', file_upper)
                n_file=filt_candy_cane(file_upper)
                if n_file is not None:
                    candy_cane.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                     print(file, '-->', n_file)
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*ARCH*NII'):
                #    
                # 1. the inconsidency happens at the missing venc number.
                #
#                 print(file_upper)
                n_file = filt_arch(file_upper)
                if n_file is not None:
                    arch.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                     print(file, '-->', n_file)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*PROBE*NII'):
#                 print(file_upper)
                n_file = filt_probe(file_upper)
                if n_file is not None:
                    probe.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*NII'):
#                 print('\t', file_upper)
                pass
            else:
#                 print('\t\t', file_upper)
                pass

#     print(len(t1))
#     for pair in t1:
#         print(pair)
#     print(len(flair))
#     for pair in flair:
#         print(pair)
#     print(len(t2))
#     for pair in t2:
#         print(pair)
#     print(len(bold))
#     for pair in bold:
#         print(pair)
#     print(len(asl))
#     for pair in asl:
#         print(pair)
#     print(len(swan))
#     for pair in swan:
#         print(pair)
#     print(len(csf))
#     for pair in csf:
#         print(pair)
#     print(len(aorta))
#     for pair in aorta:
#         print(pair)
#     print(len(cbf))
#     for pair in cbf:
#         print(pair)
#     print(len(candy_cane))
#     for pair in candy_cane:
#         print(pair)
#     print(len(arch))
#     for pair in arch:
#         print(pair)
#     print(len(probe))
#     for pair in probe:
#         print(pair)

#     print("count: ", count)
#     print(change)

In [101]:
def main():
#     source_dir=input("Enter source dir:")
#     source_dir="/Users/baymac/Desktop/Project_NaCl_Test_DataSet/3T_Last_updated_31OCT19KW"
    source_dir = '/media/ke/8tb_part2/FSL_work/all_info'
    get_path(source_dir)
        

In [102]:
if __name__ =="__main__":
    main()

['WH1331', 'CINE', 'CAROTID', 'VENC', '100', '20190514132222']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00019']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00003']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00002']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00012']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00013']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00008']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00009']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00006']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00004']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00010']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00014']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00005']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00016']
['WH1331', 'PJN', 'CAROTID', 'MRA', '20190514132222', 'I00017']
['WH1331', 'PJN', 'CAROTID', 'MRA', '2019

['WH1720', 'CINE', 'CAROTID', '20190923161210']
['SH064', 'CINE', 'CAROTID', '20190823090723']
['SH002', 'CINE', 'CAROTID', '20190816150337']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00034']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00019']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00015']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00005']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00024']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00007']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00017']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00026']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00031']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00037']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00028']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00016']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00011']
['WH1232', 'RIGHT', 'CAROTID', '20190311094507', 'I00008']
['WH1232', 'RIGHT', 'CAROTID', '